In [ ]:
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex6 import *
print("Setup Complete")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [ ]:
from xgboost import XGBRegressor

my_model_1 = XGBRegressor(random_state=0) 

my_model_1.fit(X_train,y_train)


In [ ]:
from sklearn.metrics import mean_absolute_error

predictions_1 = my_model_1.predict(X_valid) # Your code here


In [ ]:
mae_1 = mean_absolute_error(predictions_1,y_valid) # Your code here

print("Mean Absolute Error:" , mae_1)



In [ ]:
my_model_2 = XGBRegressor(n_estimators=2000,learning_rate=0.09,n_jobs=4) # Your code here

my_model_2.fit(X_train,y_train,early_stopping_rounds=20,eval_set=[(X_valid,y_valid)],verbose=False)

predictions_2 = my_model_2.predict(X_valid) # Your code here

mae_2 = mean_absolute_error(predictions_2,y_valid) # Your code here

print("Mean Absolute Error:" , mae_2)
preds_fin=my_model_2.predict(X_test)


In [ ]:
my_model_3 = XGBRegressor(n_estimators=1000,n_jobs=4,learning_rate=2.5)

my_model_3.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds=5,verbose=False)# Your code here

predictions_3 = my_model_3.predict(X_valid)

mae_3 = mean_absolute_error(predictions_3,y_valid)

print("Mean Absolute Error:" , mae_3)


In [ ]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_fin})
output.to_csv('submission.csv', index=False)